Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.851506
Minibatch accuracy: 6.2%
Validation accuracy: 9.4%
Minibatch loss at step 50: 1.553525
Minibatch accuracy: 25.0%
Validation accuracy: 56.9%
Minibatch loss at step 100: 1.055172
Minibatch accuracy: 56.2%
Validation accuracy: 64.0%
Minibatch loss at step 150: 0.591953
Minibatch accuracy: 75.0%
Validation accuracy: 72.3%
Minibatch loss at step 200: 0.966140
Minibatch accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 250: 1.064107
Minibatch accuracy: 68.8%
Validation accuracy: 78.7%
Minibatch loss at step 300: 0.428902
Minibatch accuracy: 87.5%
Validation accuracy: 79.2%
Minibatch loss at step 350: 0.511860
Minibatch accuracy: 93.8%
Validation accuracy: 77.6%
Minibatch loss at step 400: 0.242666
Minibatch accuracy: 100.0%
Validation accuracy: 79.9%
Minibatch loss at step 450: 0.792568
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 500: 0.820667
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    pool1 = tf.nn.max_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    bias2 = tf.nn.relu(conv2 + layer2_biases)
    pool2 = tf.nn.max_pool(bias2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.264739
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.942533
Minibatch accuracy: 18.8%
Validation accuracy: 42.2%
Minibatch loss at step 100: 1.193972
Minibatch accuracy: 50.0%
Validation accuracy: 58.5%
Minibatch loss at step 150: 0.600883
Minibatch accuracy: 75.0%
Validation accuracy: 73.2%
Minibatch loss at step 200: 1.046250
Minibatch accuracy: 68.8%
Validation accuracy: 77.4%
Minibatch loss at step 250: 1.030800
Minibatch accuracy: 68.8%
Validation accuracy: 78.6%
Minibatch loss at step 300: 0.350500
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 350: 0.472920
Minibatch accuracy: 93.8%
Validation accuracy: 79.3%
Minibatch loss at step 400: 0.321915
Minibatch accuracy: 93.8%
Validation accuracy: 81.6%
Minibatch loss at step 450: 0.868557
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.547783
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  size3 = ((image_size - patch_size + 1) // 2 - patch_size + 1) // 2
  layer3_weights = tf.Variable(tf.truncated_normal(
      [size3 * size3 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # C1 input 28 x 28
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    bias1 = tf.nn.relu(conv1 + layer1_biases)
    # S2 input 24 x 24
    pool2 = tf.nn.avg_pool(bias1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # C3 input 12 x 12
    conv3 = tf.nn.conv2d(pool2, layer2_weights, [1, 1, 1, 1], padding='VALID')
    bias3 = tf.nn.relu(conv3 + layer2_biases)
    # S4 input 8 x 8
    pool4 = tf.nn.avg_pool(bias3, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    # F6 input 4 x 4
    shape = pool4.get_shape().as_list()
    reshape = tf.reshape(pool4, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 2.624485
Minibatch accuracy: 6.2%
Validation accuracy: 10.5%
Minibatch loss at step 50: 1.846785
Minibatch accuracy: 31.2%
Validation accuracy: 45.0%
Minibatch loss at step 100: 1.246948
Minibatch accuracy: 62.5%
Validation accuracy: 60.0%
Minibatch loss at step 150: 0.720597
Minibatch accuracy: 75.0%
Validation accuracy: 68.9%
Minibatch loss at step 200: 0.942264
Minibatch accuracy: 81.2%
Validation accuracy: 72.9%
Minibatch loss at step 250: 1.160135
Minibatch accuracy: 75.0%
Validation accuracy: 73.8%
Minibatch loss at step 300: 0.555448
Minibatch accuracy: 87.5%
Validation accuracy: 76.3%
Minibatch loss at step 350: 0.755122
Minibatch accuracy: 87.5%
Validation accuracy: 72.2%
Minibatch loss at step 400: 0.358295
Minibatch accuracy: 93.8%
Validation accuracy: 77.8%
Minibatch loss at step 450: 0.728241
Minibatch accuracy: 81.2%
Validation accuracy: 75.4%
Minibatch loss at 

Validation accuracy: 86.1%
Minibatch loss at step 4350: 0.216432
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 4400: 1.061337
Minibatch accuracy: 75.0%
Validation accuracy: 85.7%
Minibatch loss at step 4450: 0.352788
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 4500: 0.539657
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 4550: 0.317179
Minibatch accuracy: 93.8%
Validation accuracy: 86.6%
Minibatch loss at step 4600: 0.528761
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Minibatch loss at step 4650: 0.803459
Minibatch accuracy: 81.2%
Validation accuracy: 85.5%
Minibatch loss at step 4700: 0.407516
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Minibatch loss at step 4750: 1.102154
Minibatch accuracy: 56.2%
Validation accuracy: 86.3%
Minibatch loss at step 4800: 0.577498
Minibatch accuracy: 81.2%
Validation accuracy: 86.6%
Minibatch loss at step 4850: 0.445564
Minibatch accuracy: 87.5%

Validation accuracy: 87.8%
Minibatch loss at step 8850: 0.019544
Minibatch accuracy: 100.0%
Validation accuracy: 86.8%
Minibatch loss at step 8900: 0.383019
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 8950: 0.327994
Minibatch accuracy: 87.5%
Validation accuracy: 87.6%
Minibatch loss at step 9000: 0.317523
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 9050: 0.311024
Minibatch accuracy: 93.8%
Validation accuracy: 87.5%
Minibatch loss at step 9100: 0.377113
Minibatch accuracy: 93.8%
Validation accuracy: 87.5%
Minibatch loss at step 9150: 0.747809
Minibatch accuracy: 68.8%
Validation accuracy: 87.2%
Minibatch loss at step 9200: 0.181501
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 9250: 0.605096
Minibatch accuracy: 75.0%
Validation accuracy: 87.7%
Minibatch loss at step 9300: 1.094515
Minibatch accuracy: 81.2%
Validation accuracy: 87.4%
Minibatch loss at step 9350: 0.265999
Minibatch accuracy: 87.5

Minibatch loss at step 13300: 0.214281
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%
Minibatch loss at step 13350: 0.100534
Minibatch accuracy: 100.0%
Validation accuracy: 88.5%
Minibatch loss at step 13400: 0.744938
Minibatch accuracy: 68.8%
Validation accuracy: 88.5%
Minibatch loss at step 13450: 0.374789
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 13500: 0.305054
Minibatch accuracy: 93.8%
Validation accuracy: 88.3%
Minibatch loss at step 13550: 0.398147
Minibatch accuracy: 93.8%
Validation accuracy: 88.6%
Minibatch loss at step 13600: 0.325283
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 13650: 0.422609
Minibatch accuracy: 87.5%
Validation accuracy: 88.1%
Minibatch loss at step 13700: 0.232460
Minibatch accuracy: 93.8%
Validation accuracy: 88.4%
Minibatch loss at step 13750: 0.743272
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 13800: 0.120377
Minibatch accuracy: 93.8%
Validation acc

Minibatch loss at step 17750: 0.166723
Minibatch accuracy: 93.8%
Validation accuracy: 89.0%
Minibatch loss at step 17800: 0.592235
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 17850: 0.364704
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 17900: 0.326610
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 17950: 0.487085
Minibatch accuracy: 81.2%
Validation accuracy: 88.7%
Minibatch loss at step 18000: 0.258592
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 18050: 0.328145
Minibatch accuracy: 87.5%
Validation accuracy: 88.8%
Minibatch loss at step 18100: 0.730799
Minibatch accuracy: 81.2%
Validation accuracy: 89.1%
Minibatch loss at step 18150: 0.144687
Minibatch accuracy: 93.8%
Validation accuracy: 88.9%
Minibatch loss at step 18200: 0.411307
Minibatch accuracy: 93.8%
Validation accuracy: 89.0%
Minibatch loss at step 18250: 0.610345
Minibatch accuracy: 75.0%
Validation accu